In [7]:
import openai 
from openai import OpenAI

OPENAI_API_KEY = 'None'
openai.api_key = OPENAI_API_KEY

import pandas as pd

client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [2]:
final_df = pd.read_excel('merged_file.xlsx')

In [3]:
def extract_classification_and_reasoning(response):
    # Split the response by lines
    lines = response.split('\n')
    
    # Extract classification
    classification_line = lines[0]
    classification = classification_line.replace("Classification: ", "").strip()
    
    # Extract reasoning
    reasoning_line = lines[1]
    reasoning = reasoning_line.replace("Reasoning: ", "").strip()
    
    return classification, reasoning

In [6]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used ChatGPT.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'. 'Relevant' means the author explicitly mentions how they personally used ChatGPT, like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [5]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used ChatGPT.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'. 'Relevant' means the author explicitly mentions how they personally used ChatGPT, like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [4]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [10]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
The post mentions knowing about developers using ChatGPT but does not explicitly state that the author personally used ChatGPT. (Score: 9)
The author explicitly mentions using ChatGPT to help with the design side of web development, specifically to "pretty things up." (Score: 10)
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress dev

in the above prompt:


Number of mistakes:
5

Number of false positives:
2

Number of false negatives:
3

Number of relevant:
8

Number of irrelevant:
22


number of score = 9: 14 

number of score = 10: 16


when there is a mistake:

1 time the score was 10 

4 times the score was 9


In [12]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT .\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
The author mentions knowing about developers using ChatGPT but does not explicitly state how they personally used ChatGPT. Score: 9
The author explicitly mentions using ChatGPT to help with the design side of web development, specifically to "pretty things up." (Score: 10)
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress developmen

In [13]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
The post mentions developers using ChatGPT in their portfolios but does not explicitly state how the author personally used ChatGPT or similar AI for their work. Score: 9
The author explicitly mentions using ChatGPT to help with the design side of web development, indicating a personal use of the AI tool in their work. Score: 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make m

intersting prompt:

In [15]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt\n"
    prompt = prompt + "try to classify all the relevant posts as relevant, its better to have false positives than false negatives\n\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            print(reason)

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
The post mentions knowing about developers using ChatGPT in their portfolios but does not explicitly state that the author has used ChatGPT or similar AI themselves. (Score: 9)
The author explicitly mentions using ChatGPT to help with the design side of web development, indicating a personal use of the AI tool in their work. Score: 9
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to m

probably the best one yet:

In [21]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

also preety good prompt:

In [23]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or are using ChatGPT or similar AI tools.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'.\n\n"
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT or similar AI tools for their work, even if the usage is minor. Examples include statements like 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also include indirect mentions of using ChatGPT or similar AI tools, such as a post written with the assistance of ChatGPT or someone seeking advice on how to improve their usage of ChatGPT.\n\n"
    prompt = prompt + "Briefly explain your reasoning in one sentence and include a confidence score(in your reason) from 1 to 10, where 10 means very sure and 1 means very unsure.\n\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [30]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [35]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who is following instructions provided by chatgpt\n"
    prompt = prompt + "Take into considiration that this is a post that was posted in reddit and can contain slang or non formal language\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Also if your confidance isnt 10 also use the post title combined with the information from the post to make your decision\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

prompt with 0 false negatives almost always:

In [37]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who created Ai himself\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

seems like a good prompt also: best if looking at amount of overall mistakes

In [38]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who created Ai himself\n"
    prompt = prompt + "you should still be strict and mark as relevant only posts that are directly related to the use of chatgpt or similar Ai for example if someone says that he heard that chatgpt is good in somekind of task but doesnt mantion he himself used it it isnt a relevant post\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

probably the best prompt but maybe to specifiec:

In [41]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.' or 'chatgpt suggested i write it on my own'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who created Ai himself or even someone who follows chatgpt instructions to do something\n"
    prompt = prompt + "you should still be strict and mark as relevant only posts that are directly related to the use of chatgpt or similar Ai for example if someone says that he heard that chatgpt is good in somekind of task but doesnt mantion he himself used it it isnt a relevant post\n"
    prompt = prompt + "Briefly explain your reasoning (one sentence).in the same sentance include a score of 1-10 of how much you sure about the classification(10 is very sure and 1 is very not sure)\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant\nReasoning: Your Reason\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification, reasoning = extract_classification_and_reasoning(response_content)
    return classification, reasoning

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification, reason = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification, 'Reason': reason})
                print(reason)
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [9]:
def extract_classification(response):
    # Split the response by lines
    lines = response.split('\n')
    
    # Extract classification
    classification_line = lines[0]
    classification = classification_line.replace("Classification: ", "").strip()
    
    return classification

In [10]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.' or 'chatgpt suggested i write it on my own'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who created Ai himself or even someone who follows chatgpt instructions to do something\n"
    prompt = prompt + "you should still be strict and mark as relevant only posts that are directly related to the use of chatgpt or similar Ai for example if someone says that he heard that chatgpt is good in somekind of task but doesnt mantion he himself used it it isnt a relevant post\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DONT RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [6]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author specifically mentions how they used or using ChatGPT / similar AI.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar AI for their work (even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPThelps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n"
    prompt = prompt + "Relevant posts can also discuss the use of ChatGPT/another AI in a non direct way, for example someone who wrote a post using ChatGPT or someone who looks for help on how to improve his usage of ChatGPT or someone who created AI themselves or even someone who follows ChatGPT instructions to complete a task.\n"
    prompt = prompt + "You should still be strict and mark as relevant only posts that are directly related to the use of ChatGPT or another AI. For example, if someone says that they heard that ChatGPT is good in some type of task but does not mention using it themselves, the post is not relevant.\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DON’T RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT.\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

In [5]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author specifically mentions how they used or using ChatGPT / similar AI.\n\n"

    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."

    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar AI for their work (even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.' or 'chatgpt suggested i write it on my own'\n\n"

    prompt = prompt + "Relevant posts can also discuss the use of ChatGPT/another AI in a non direct way, for example someone who wrote a post using ChatGPT or someone who looks for help on how to improve his usage of ChatGPT or someone who created AI themselves or even someone who follows ChatGPT instructions to complete a task.\n"

    prompt = prompt + "You should still be strict and mark as relevant only posts that are directly related to the use of ChatGPT or similar AI. For example, if someone says that they heard that ChatGPT is good in some type of task but does not mention using it themselves, the post is not relevant.\n"

    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DON’T RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT.\n"


    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        print("testing on last 10")
        examples = []
        for i, row in final_df[0:20].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[20:].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 1:
        examples = []
        print("testing on middle 10")

        for i, row in final_df[20:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })
        for i, row in final_df[0:10].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[10:20].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1

    if j == 2:
        print("testing on first 10")
        examples = []
        for i, row in final_df[10:].iterrows():
            classification = row['Final'].strip().lower()
            examples.append({
                "post": row['content'],
                "classification": classification,
                "title": row['title']
            })

        # Classify all posts and store the results
        results = []
        for index, row in final_df[0:10].iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})

            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

testing on last 10
{'ID': '14t716p', 'Post': "Hello!\n\nI am a web developer and I have been studying for about 3 years. I can build SPA's and MPA's in Golang pretty quickly and have a good work flow. I love HTMX and using it has been a huge productivity boost. Anyways, I am confident most basic CRUD app functionality is within my reach. I do struggle with the design side of things, but I've found tools like Chatgpt help to pretty things up. I like tailwind but can do basic css, It'll just take me a bit longer to get media queries and responsiveness put together.\n\nAll of this to be said, I want to get into freelancing. I love coding and I would love to make money on the side doing it.\n\nI hear wordpress development is highly sought after in freelance, but I really just want to hear the opinion of someone who actually freelances.\n\nWhat stack did you find most productive and go your going in the field?", 'title': 'Freelancing | Best stack to build things fast and get going?', 'Class

New 30:

In [2]:
final_df_new = pd.read_excel('New 30.xlsx')

In [28]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author mentions specifically how they used or using ChatGPT / similar Ai.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar Ai for their work(even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.' or 'chatgpt suggested i write it on my own'\n\n"
    prompt = prompt + "Relevant posts can also be a post that discusses the use of ChatGPT/Similar Ai in a non direct way, for example someone who wrote a post using chat gpt or someone who looks for help on how to improve his usage of chat gpt or someone who created Ai himself or even someone who follows chatgpt instructions to do something\n"
    prompt = prompt + "you should still be strict and mark as relevant only posts that are directly related to the use of chatgpt or similar Ai for example if someone says that he heard that chatgpt is good in somekind of task but doesnt mantion he himself used it it isnt a relevant post\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DONT RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        
        # Classify all posts and store the results
        results = []
        for index, row in final_df_new.iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

{'ID': '13z2z9f', 'Post': 'I asked Chatgpt this problem of mine and I got two different answer so If anyone can confirm which one is the correct one according to IFRS standard, from my accounting textbook, supposedly 2nd answer is the correct one but my teacher is an asshole he doesn\'t reply to my messages\n\nQ:"Company A is both our supplier and customer for our automobile service company.\n\nOur company currently have due amount of account payable to Company A, then we provided services to Company A, but instead of paying for the services provided by us, Company A want us to deduct the total amount of services provided from the amount due in account payable.\n\nHow am I supposed to record this?\n\nPlease walk me through the recording process from journals to ledger"\n\n1st answer:\nTo record the transaction where your company provides services to Company A and they want you to deduct the total amount of services provided from the accounts payable balance, you would follow these step

In [29]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post, examples):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analyzing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author specifically mentions how they used or using ChatGPT / similar AI.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar AI for their work (even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPThelps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n"
    prompt = prompt + "Relevant posts can also discuss the use of ChatGPT/another AI in a non direct way, for example someone who wrote a post using ChatGPT or someone who looks for help on how to improve his usage of ChatGPT or someone who created AI themselves or even someone who follows ChatGPT instructions to complete a task.\n"
    prompt = prompt + "You should still be strict and mark as relevant only posts that are directly related to the use of ChatGPT or another AI. For example, if someone says that they heard that ChatGPT is good in some type of task but does not mention using it themselves, the post is not relevant.\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DON’T RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT.\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        
        # Classify all posts and store the results
        results = []
        for index, row in final_df_new.iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post, examples)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

{'ID': '13z2z9f', 'Post': 'I asked Chatgpt this problem of mine and I got two different answer so If anyone can confirm which one is the correct one according to IFRS standard, from my accounting textbook, supposedly 2nd answer is the correct one but my teacher is an asshole he doesn\'t reply to my messages\n\nQ:"Company A is both our supplier and customer for our automobile service company.\n\nOur company currently have due amount of account payable to Company A, then we provided services to Company A, but instead of paying for the services provided by us, Company A want us to deduct the total amount of services provided from the amount due in account payable.\n\nHow am I supposed to record this?\n\nPlease walk me through the recording process from journals to ledger"\n\n1st answer:\nTo record the transaction where your company provides services to Company A and they want you to deduct the total amount of services provided from the accounts payable balance, you would follow these step

In [10]:
import pandas as pd
import json

# Function to classify a single post
def classify_post(post):
    prompt = f"CONTEXT: As part of a study on how people perceive others using ChatGPT in their jobs, I am analysing posts from subreddits related to various professions (e.g., accounting, web development). The focus is on posts where the author specifically mentions how they used or using ChatGPT / similar AI.\n\n"
    prompt = prompt + f"YOUR TASK: Given the following post, classify it as 'relevant' or 'irrelevant'."
    prompt = prompt + "'Relevant' means the author explicitly mentions how they personally used ChatGPT/Similar AI for their work (even if they used it in a small or not significant way), like in these examples: 'I use ChatGPT for summarizing my notes,' 'ChatGPT helps me draft emails more efficiently,' or 'Using ChatGPT, I automate routine data analysis.'\n"
    prompt = prompt + "Relevant posts can also discuss the use of ChatGPT/another AI in a non direct way, for example someone who wrote a post using ChatGPT or someone who looks for help on how to improve his usage of ChatGPT or someone who created AI themselves or even someone who follows ChatGPT instructions to complete a task.\n"
    prompt = prompt + "You should still be strict and mark as relevant only posts that are directly related to the use of ChatGPT or another AI. For example, if someone says that they heard that ChatGPT is good in some type of task but does not mention using it themselves, the post is not relevant.\n"
    prompt = prompt + "Return the answer in this format: Classification: Irrelevant or Relevant. DON’T RETURN ANYTHING THAT IS NOT RELEVANT OR IRRELEVANT.\n"

    #for example in examples:    
        #prompt += f"Example: Title: {example['title']}\nPost: {example['post']}\nClassification: {example['classification']}\n\n"
    prompt += f"The Post to tag:"
    prompt += post

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    response_content = response.choices[0].message.content.strip()
    classification = extract_classification(response_content)
    return classification

mistake_counter = 0
false_positives = 0
false_negatives = 0
irelevant_counter = 0
relevant_counter = 0

for j in range(0, 3):
    if j == 0:
        
        # Classify all posts and store the results
        results = []
        for index, row in final_df_new.iterrows():
            post = "title:" + row['title'] + ", post:" +  row['content']
            classification = classify_post(post)
            results.append({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
            if(classification == 'Irrelevant'):
                irelevant_counter += 1
            else:
                relevant_counter += 1    

            if(row['Final'] != classification):
                mistake_counter += 1
                print({'ID': row['id'], 'Post': row['content'], 'title': row['title'], 'Classification': classification})
                if(classification == 'Irrelevant'):
                    false_negatives += 1
                    print("the row above is false negative")
                else:
                    false_positives += 1


print("Number of mistakes:")
print(mistake_counter)
print("Number of false positives:")
print(false_positives)
print("Number of false negatives:")
print(false_negatives)
print("Number of relevant:")
print(relevant_counter)
print("Number of irrelevant:")
print(irelevant_counter)

{'ID': '13z2z9f', 'Post': 'I asked Chatgpt this problem of mine and I got two different answer so If anyone can confirm which one is the correct one according to IFRS standard, from my accounting textbook, supposedly 2nd answer is the correct one but my teacher is an asshole he doesn\'t reply to my messages\n\nQ:"Company A is both our supplier and customer for our automobile service company.\n\nOur company currently have due amount of account payable to Company A, then we provided services to Company A, but instead of paying for the services provided by us, Company A want us to deduct the total amount of services provided from the amount due in account payable.\n\nHow am I supposed to record this?\n\nPlease walk me through the recording process from journals to ledger"\n\n1st answer:\nTo record the transaction where your company provides services to Company A and they want you to deduct the total amount of services provided from the accounts payable balance, you would follow these step